# 目的
遺伝子学的分類に基づいた、再発の2値分類を実施する。  

In [74]:
# 基本的なライブラリ
import itertools
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# パイプライン
from imblearn.pipeline import Pipeline
from imblearn import FunctionSampler
from sklearn.base import BaseEstimator, TransformerMixin

# サンプリング
from imblearn.over_sampling import SMOTE

# モデル
from sksurv.ensemble import RandomSurvivalForest

# 評価指標
from sklearn.metrics import mean_squared_error

# local libraries
import config
import functions

SEED = config.SEED
functions.fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

In [2]:
df_recurrenced = pd.read_pickle(
    config.INTERIM_PREPROCESSED_RECURRENCE_DIR + "/df_recurrenced4numeric.pkl"
)

# データ全体での予測モデル検証

## ベースモデルの学習結果

各特徴量選択されたdfを用いて基本的な2値分類モデルを学習する

In [101]:
def smote4survival(X, y):
    _y = pd.DataFrame.from_records(y)
    _y_rm = _y.RFS_STATUS
    _y_rs = _y.RFS_MONTHS

    X_resample, y_rm_resample = SMOTE(random_state=config.SEED).fit_resample(X, _y_rm)
    X_resample, y_rs_resample = SMOTE(random_state=config.SEED).fit_resample(X, _y_rs)
    y_resample = pd.merge(
        y_rm_resample, y_rs_resample, right_index=True, left_index=True
    ).to_records(index=False)

    return X_resample, y_resample

In [102]:
def validate_models(
    model_order: dict = {"original_data": []},
    input_file_path: str = None,
    output_file_path: str = None,
):

    cat_path = "/val_set"
    try:
        _X_train = pd.read_pickle(input_file_path + cat_path + "/train/X.pkl")
        _y_train = pd.read_pickle(
            input_file_path + cat_path + "/train/y.pkl"
        ).to_records(index=False)
        _X_val = pd.read_pickle(input_file_path + cat_path + "/val/X.pkl")
        _y_val = pd.read_pickle(input_file_path + cat_path + "/val/y.pkl").to_records(
            index=False
        )
    except:
        # pkl file dose not exist
        return

    display(_X_train.shape, _y_train.shape)

    # 学習用パイプラインの構築
    # パイプラインメソッドの関係上、オーバーサンプリングもこちらで行う
    # ※imblearnではオーバーサンプリング後のデータの保存が難しい
    # こちらはimblearnのPipelineなので注意
    _preprocess_pipeline = Pipeline(steps=model_order)
    _preprocess_pipeline.fit(_X_train, _y_train)
    _y_train_pred = _preprocess_pipeline.predict(_X_train)
    _y_val_pred = _preprocess_pipeline.predict(_X_val)
    print(mean_squared_error(_y_val.RFS_MONTHS, _y_val_pred, squared=False))

In [105]:
# 層別化する特徴量
subgroup_columns = [
    "CLAUDIN_SUBTYPE",
    # "THREEGENE",
]
# 遺伝子発現量以外の特徴量を削除
drop_columns = [
    "CLAUDIN_SUBTYPE",
    "THREEGENE",
]

# 実施したい学習の定義
model_order_dicts = [
    {
        "model_name": "RandomSurvivalForest",
        "preprocess_name": "preprocess4numeric_1",
        "model_order": [
            # ("smote", FunctionSampler(func=smote4survival)),# 目的変数が連続値なので、SMOTEは実施しない
            (
                "RandomSurvivalForest",
                RandomSurvivalForest(
                    n_estimators=1000,
                    min_samples_split=10,
                    min_samples_leaf=15,
                    n_jobs=-1,
                    random_state=config.SEED,
                ),
            ),
        ],
    },
]

for subgroup_column in subgroup_columns:  # 各サブグループへの適用
    print("##########" * 10)
    print("subgroup_column: ", subgroup_column)
    for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループの要素毎への適用
        print("==========" * 10)
        print("subgroup: ", subgroup)
        for model_order_dict in model_order_dicts:
            print(".........." * 10)
            print("model_order_dict name: ", model_order_dict["model_name"])
            input_file_path = "./{0}/{1}/{2}/{3}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_DIR,
                subgroup_column,
                subgroup,
                model_order_dict["preprocess_name"],
            )
            """
            output_file_path = "./{0}/{1}/{2}/{3}".format(
                config.INTERIM_MODELS_RECURRENCE_DIR,
                subgroup_column,
                subgroup,
                model_order_dict["model_name"],
            )"""
            validate_models(
                model_order=model_order_dict["model_order"],
                input_file_path=input_file_path,
            )

####################################################################################################
subgroup_column:  CLAUDIN_SUBTYPE
subgroup:  claudin-low
....................................................................................................
model_order_dict name:  RandomSurvivalForest


(159, 140)

(159,)

145.6370641642193
subgroup:  LumA
....................................................................................................
model_order_dict name:  RandomSurvivalForest


(544, 475)

...

(113,)

148.55229430129737
subgroup:  Basal
....................................................................................................
model_order_dict name:  RandomSurvivalForest


(161, 144)

(161,)

144.70662569359638
subgroup:  NC
....................................................................................................
model_order_dict name:  RandomSurvivalForest
